In [114]:
# for the twitter section
import tweepy
import os
import datetime
import re
from pprint import pprint
import shutil

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [115]:
artists = {'eminem':"https://www.azlyrics.com/e/eminem.html",
           'fiddy':"https://www.azlyrics.com/19/50cent.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

In [116]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items() :
    print(f'Scraping {artist}')
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())
    
    soup = BeautifulSoup(r.content, "html.parser")

    lyric_divs = soup.find_all("div", class_="listalbum-item")

    lyric_urls = []

    for ld in lyric_divs[:20]:
        this_anchor = ld.find('a')
        lyric_urls+=['https://www.azlyrics.com'+this_anchor['href']]

    lyrics_pages[artist]=lyric_urls
    print(f'Done Scraping {artist}')
    # now extract the links to lyrics pages from this page
    # store the links `lyrics_pages` where the key is the artist and the
    # value is a list of links. 
    

Scraping eminem
Done Scraping eminem
Scraping fiddy
Done Scraping fiddy


In [117]:
print(lyrics_pages)

defaultdict(<class 'list'>, {'eminem': ['https://www.azlyrics.com/lyrics/eminem/infinite.html', 'https://www.azlyrics.com/lyrics/eminem/wegointerlude.html', 'https://www.azlyrics.com/lyrics/eminem/itsok.html', 'https://www.azlyrics.com/lyrics/eminem/313.html', 'https://www.azlyrics.com/lyrics/eminem/tonight.html', 'https://www.azlyrics.com/lyrics/eminem/maxine.html', 'https://www.azlyrics.com/lyrics/eminem/openmic.html', 'https://www.azlyrics.com/lyrics/eminem/never2far.html', 'https://www.azlyrics.com/lyrics/eminem/searchin.html', 'https://www.azlyrics.com/lyrics/eminem/backstabber.html', 'https://www.azlyrics.com/lyrics/eminem/jealousywoesii.html', 'https://www.azlyrics.com/lyrics/eminem/introslimshady.html', 'https://www.azlyrics.com/lyrics/eminem/lowdowndirty.html', 'https://www.azlyrics.com/lyrics/eminem/ifihad.html', 'https://www.azlyrics.com/lyrics/eminem/justdontgiveafuck.html', 'https://www.azlyrics.com/lyrics/eminem/mommyskit.html', 'https://www.azlyrics.com/lyrics/eminem/jus

In [118]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For eminem we have 20.
The full pull will take for this artist will take 0.06 hours.
For fiddy we have 20.
The full pull will take for this artist will take 0.06 hours.


In [119]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)

In [120]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics/") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [121]:
url_stub = "https://www.azlyrics.com" 
start = time.time()

total_pages = 0 

for artist in lyrics_pages :
    pass
    # Use this space to carry out the following steps: 
    
    # 1. Build a subfolder for the artist
    if os.path.isdir(f"lyrics/{artist}") : 
        shutil.rmtree(f"lyrics/{artist}")

    os.mkdir(f"lyrics/{artist}")

    # 2. Iterate over the lyrics pages (20)

    for page in lyrics_pages[artist][:20]:
    
    # 3. Request the lyrics page. 

        r = requests.get(page)
        time.sleep(5 + 10*random.random())

    # 4. Extract the title and lyrics from the page.

        soup = BeautifulSoup(r.content, "html.parser")

    # 4. Extract the title and lyrics from the page.

        title = str(soup.title.string).split("Lyrics")[0] # Get lyrics names from pagetitle

        divs = soup.find_all("div")

        lyrics = ''

        for div in divs:
            for c in div.contents:
                if "Usage of azlyrics.com" in c:
                    lyrics = (div.text)
                    break

        filename = generate_filename_from_link(page)
        with open(os.path.join('.','lyrics',artist,filename), 'w') as f:
            f.write(title)
            # already two spaces but this will write two spaces: f.write('\n\n')
            f.write(lyrics)
        

    # 5. Write out the title, two returns ('\n'), and the lyrics. Use `generate_filename_from_url`
    #    to generate the filename. 
    
    # Remember to pull at least 20 songs per artist. It may be fun to pull all the songs for the artist

In [122]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.12 hours.


In [123]:
## Validation

def words(text): 
    return re.findall(r'\w+', text.lower())

artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")

For eminem we have 20 files.
For eminem we have roughly 13425 words, 2518 are unique.
For fiddy we have 20 files.
For fiddy we have roughly 14519 words, 2107 are unique.
